# Cipher Tools Library
A collection of tools developed along the way to help with deciphering

In [ ]:
import string

# A simple translation method.
# text: The text to translate, in upper case
# key: A substitution alphabet, usually in lower case so that the translated characters show up
def decode(text, key):
    table = str.maketrans(string.ascii_uppercase, key)
    print(text.upper().translate(table))

In [ ]:
import string

# Alternative version of the above, but this time preserving the case
def decode2(text, key):
    table = str.maketrans(string.ascii_letters, key.lower()+key.upper())
    print(text.translate(table))

In [ ]:
from collections import Counter

# Takes a string or list of items and counts the frequencies of those items
# data: The list or string to analyse
# max_values: The maximum number of values to display (set to None for no limit)
# no_columns: The amount of columns to use in the output
def frequency_analysis(data, max_values=30, no_columns=5):
    frequencies = Counter()
    for item in data:
        frequencies[item] += 1
    
    total = sum(frequencies.values())
    column = 1
    for item, frequency in frequencies.most_common(max_values):
        print(f"{item}: {frequency:2} ({frequency / total:.2%})", end=" " if column % no_columns else "\n")
        column += 1
    print("\n-----")

In [ ]:
# A method to decode/encode a Vigenere cipher.  Case and punctuation are preserved.
# text: The text to encode
# key: The key to use
# encode: If set to True, will encode the given text instead of decoding it
def vigenere(text, key, encode=False):
    shifts = [ord(letter.upper()) - ord("A") for letter in key]
    key_length = len(key)
    key_index = 0
    for letter in text:
        letter_val = ord(letter)
        if letter.isupper():
            offset = ord("A")
        elif letter.islower():
            offset = ord("a")
        else:
            print(letter, end="")
            continue

        if encode:
            shift = shifts[key_index]
        else:
            shift = -shifts[key_index]

        print(chr((((letter_val - offset) + shift) % 26) + offset), end="")
        key_index = (key_index + 1) % key_length
    print()

In [ ]:
import string

# Attempts to determine the keyword used for a substition cipher
# alphabet: The substitution alphabet
def decode_key(alphabet):
    decoded_alphabet = ""
    for letter in string.ascii_lowercase:
        decoded_alphabet += chr(alphabet.find(letter) + ord("a"))
    print(f"Decoded alphabet: {decoded_alphabet}")
    
    remaining_letters = list(string.ascii_lowercase)
    for pos, letter in enumerate(decoded_alphabet):
        remaining_letters.remove(letter)
        next_letter_index = remaining_letters.index(decoded_alphabet[pos+1])
        if decoded_alphabet[pos+1:] == "".join(remaining_letters[next_letter_index:] + remaining_letters[:next_letter_index]):
            print(f"Keyword: {decoded_alphabet[:pos+2]}")
            break

# Creates a substitution alphabet from a given keyword
def encode_key(keyword):
    key = list("." * 26)
    letters = list(string.ascii_lowercase)
    position = 0
    for letter in keyword:
        position = ord(letter)-ord("a")
        key[position] = letters.pop(0)
    index = (position + 1) % 26
    while index != position:
        if key[index] == ".":
            key[index] = letters.pop(0)
        index = (index + 1) % 26

    print("".join(key))


In [ ]:
# Tools to help decode Vigenere ciphers


# A function that searches for multiple ocurrences of letters of the specified size and prints out the most common ones
def pattern_finder(text, size):
    patterns = Counter()
    for index in range(len(text)):
        patterns[text[index:index+size]] += 1

    column = 0
    for item, pattern in patterns.most_common(30):
        print(f"{item}: {pattern:2}", end=" ")
        column += 1
        if column % 5 == 0:
            print()


# A function that searches the given text for a word and figures out where it would line up with a key of a given size
def get_alignment(text, word, key_size):
    print(f"{word}/{key_size}", end=" ")
    alignments = []
    position = -1
    while True:
        position = text.find(word, position+1)
        if position >= 0:
            alignment = position % key_size
            print(f"{position} ({alignment})", end=" ")
            alignments.append(alignment)
        else:
            break
    if alignments.count(alignments[0]) == len(alignments):
        print("All the same!", end="")
    print()


# A function to reverse search for a key that would convert the given word to an encoded word
def lookup_key(word, encoded_word):
    key = ""
    for letter, encoded_letter in zip(word, encoded_word):
        diff = (ord(encoded_letter) - ord(letter)) % 26
        key += chr(diff + ord('A'))
    print(key)

In [ ]:
def columnar_transposition(text, key):
    text = text.replace(" ", "")
    columns = dict()
    column_length = len(text) // len(key)
    count = 0
    for column in sorted(key):
        start_pos = count*column_length
        columns[column] = text[start_pos:start_pos+column_length]
        count += 1
    for row in range(column_length):
        for char in key:
            print(columns[char][row], end="")


In [1]:
# Remove duplicate (and non-alpabetic) letters from a word and return the result
def remove_duplicates(word):
    result = ""
    for letter in word:
        if letter.isalpha() and letter not in result:
            result += letter
    return result


# Search some text for a word that contains letters in a given order
# text: The test to search
# order: A list of integers representing the alphabetical order of the letters in the word
def search_combinations(text, order):
    for word in text.split():
        word = remove_duplicates(word)
        if len(word) == len(order):
            positions = dict()
            for n, letter in enumerate(sorted(word)):
                positions[letter]=n
            if [positions[letter] for letter in word] == order:
                print(f"Found it: {word}")
                break


# A couple of tests
search_combinations("nazis sword reich", [4, 1, 3, 0, 2])

message = """Phil,
Sorry I haven’t been in touch much, Churchill asked BOSS to set up an operations wing in the UK under the name of the Special Operations Executive and that has occupied a lot of my time. As soon as we got set up I was put in touch with Einar Skinnarland, an engineer from Vemork who had hijacked a coastal steamer and sailed to Aberdeen to join the war effort here. Churchill ordered us to work up plans to attack the plant and Einar helped us to brief an intelligence gathering team to infiltrate the region. Operation Grouse was launched in October with an advance party of four officers and NCOs led by Jens-Anton Poulsson. They were parachuted into the Hardangervidda as German patrols tended to avoid it, and after a period of observation they prepared the ground for a glider assault. Under the codename Operation Freshman we sent over two gliders carrying commandos equipped with explosives and everything they needed to effect an escape, but a combination of bad weather and bad luck killed the mission. Both gliders made it to the Norwegian coast, but one crashed early on, and the other in the mountains. We were not aware of survivors, and unfortunately the Germans now knew that the plant was a target and stepped up security. They lit up the place with floodlights, mined the approaches and, for a while, stepped up the guard rotas. Grouse volunteered to stay in place, changing their callsign to Swallow and continued to send intelligence reports. They reported that although the mines and lights were still in place there were signs that security was beginning to slacken.
With these updates we decided to try again, and launched Operation Gunnerside. Six Norwegian commandos led by Joachim Ronnenberg were parachuted in from an RAF Halifax and joined up with Swallow. The attached document is their mission report. They sent it from the plateau while retreating from the plant in case they didn’t make it back, so have used a standard combination of basic ciphers to make it hard to crack but easy to implement. In training we recommended a combination of Casear shift and basic transposition. I leave it to you to decipher."""

search_combinations(message, [3,0,6,4,2,1,5])

Found it: reich
Found it: patrols
